In [1]:
# Import required libraries
import os, time, requests
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

# robust: search upward for the nearest .env
env_path = find_dotenv(usecwd=True) or (Path(__file__).resolve().parent / ".env")
load_dotenv(env_path, override=False)

True

In [2]:
print("CWD:", Path.cwd())

CWD: /Users/mac/Learningnewthings/trading-pipeline/python


In [3]:
# Get API key from environment variable
TD_KEY = os.getenv("TWELVE_DATA_KEY")
assert TD_KEY, "TWELVE_DATA_KEY not set"

In [ ]:
def latest_minute_bars_twelvedata(symbols):
    """
    Pull 1 latest bar for many symbols in ONE call (good for prototypes).
    NOTE: A 50-symbol batch ~= 50 credits on free plan (8/min, 800/day).
    """
    base = "https://api.twelvedata.com/time_series"
    params = {
        "symbol": ",".join(symbols),      # up to ~120 symbols
        "interval": "1min",
        "outputsize": 1,                  # latest bar only
        "timezone": "UTC",
        "apikey": TD_KEY
    }
    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    
    if isinstance(data, dict) and data.get("status") == "error":
        raise RuntimeError(f"Twelve Data error: {data.get('message')}")
    
    # Normalize to a list of dicts: symbol, ts, ohlcv
    rows = []
    
    def _row(sym, v):
        return {
            "symbol": sym,
            "event_time": v["datetime"],   # already UTC if timezone=UTC
            "o": float(v["open"]),
            "h": float(v["high"]),
            "l": float(v["low"]),
            "c": float(v["close"]),
            "v": float(v["volume"]),
            "source": "twelvedata",
        }
        
    
    if "values" in data and "meta" in data:
        vals = data.get("values", [])
        if vals:
            sym = data["meta"]["symbol"]
            rows.append(_row(sym, vals[0]))
        return rows
    
    for sym in symbols:
        payload = data.get(sym)
        if not isinstance(payload, dict):
            continue
        if payload.get("status") == "error":
            print(f"Twelve Data error for {sym}: {payload.get('message')}")
            continue
        vals = payload.get("values", [])
        if vals:
            rows.append(_row(sym, vals[0]))
    
    return rows

In [5]:
latest_minute_bars_twelvedata(["AAPL"])

[]